<a href="https://colab.research.google.com/github/Laredo-Labs/atom-plugin/blob/main/notebooks/demo_cases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets accelerate more-itertools unidiff optuna

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/laredo/llm-research/src')
sys.path.append('/content/drive/MyDrive/laredo/llm-research/src/eval')
import commit_tester

In [ ]:
from transformers import GPT2TokenizerFast, GPT2LMHeadModel
import termcolor
import pandas as pd
import torch
import time

model_path = "/content/drive/MyDrive/laredo/models/background-20220330"
dataset_path = "/content/drive/MyDrive/laredo/data/20220330-165311-unified"
dataset_split = "validation"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer = GPT2TokenizerFast.from_pretrained("/content/drive/MyDrive/laredo/tokenizer")
model = GPT2LMHeadModel.from_pretrained(model_path)
model = model.to(device)

print(f"Starting demo case search of {model_path} with {dataset_path}[{dataset_split}]")

results = commit_tester.test_dataset(model, tokenizer, dataset_path, dataset_split)

In [ ]:
# sort by matched token ratio and show top 5
results.sort(key=lambda test: test.matched_token_ratio(), reverse=True)
for result in results[:5]:
    commit_tester.render_result(result, tokenizer)


In [ ]:
# sort by matched tokens and show top 5
results.sort(key=lambda test: test.matched_tokens(), reverse=True)
for result in results[:5]:
    commit_tester.render_result(result, tokenizer)

In [ ]:
print(f"{tokenizer.convert_ids_to_tokens(results[0].tokens)}")

Optional testing of the results to ensure test_dataset's testing in different contexts produces results that would be consistent with what would happen normally with our client.

In [ ]:
commit_tester.corroborate_results(model, tokenizer, results, tests_per_commit=100)